In [1]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.layers import Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from sklearn.tree import DecisionTreeClassifier
from sys import getsizeof

In [6]:
def read_syms_from_txt():  
  with open("syms.txt","r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
  #'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
 'all_cryptocurrencies_us']
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   #print(len(symbols))
   #pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["sugg"],axis=1)
  y = data["sugg"]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1,random_state=55)
  print(xTrain.shape,end=" ")
  print(yTrain.shape)
  print(xTest.shape,end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def extract_data(df):
    rows = []
    for each in range(5,df.shape[0]-1):
        row = [
               df[each-5][0],df[each-5][1],df[each-5][2],df[each-5][3],
                df[each-4][0],df[each-4][1],df[each-4][2],df[each-4][3],
                df[each-3][0],df[each-3][1],df[each-3][2],df[each-3][3],
                df[each-2][0],df[each-2][1],df[each-2][2],df[each-2][3],
                df[each-1][0],df[each-1][1],df[each-1][2],df[each-1][3],
                df[each][0],df[each][1],df[each][2],df[each][3],
        ]
def process(dfs): 
   fixed_dfs = []
   for df in dfs:
      
       

In [7]:
#symbols = ["MSFT","AAPL","GOOG","TSLA","AMZN"]
#symbols = ["BTC-USD","LTC-USD","TRX-USD","XRP-USD","ETH-USD","BNB-USD","DASH-USD","VET-USD","LINK-USD","ADA-USD","DOT-USD","SOL-USD","BCH-USD","FTT-USD","FIL-USD","XMR-USD"]
#symbols = ["AAPL","MSFT","TSLA","GOOG"]
#symbols = ["BTC-USD","ETH-USD"]
#symbols = ["BTC-USD"]

symbols = read_syms_from_txt()
dfs = []
for symbol in symbols[:100]:
           data = yf.download(symbol,period="MAX",interval="1d",progress=False)
           if data.empty :
             print("Passing...")
           else:
               dfs.append(data)
ndfs = dfs
dfs = []
for df in ndfs:
        if df.shape[0] > 11:
          dfs.append(df)
data = process(dfs)
data.to_parquet("400_sym.parquet")
#df = pd.read_parquet("/content/drive/MyDrive/Colab Files/250_crypto_1d_scaled_yf.parquet")
xTrain, xTest, yTrain, yTest = spliting(data)
data


1 Failed download:
- PIH: No data found, symbol may be delisted
Passing...

1 Failed download:
- PIHPP: No data found, symbol may be delisted
Passing...

1 Failed download:
- FCCY: No data found, symbol may be delisted
Passing...

1 Failed download:
- JFK: No data found, symbol may be delisted
Passing...

1 Failed download:
- JFKKR: No data found, symbol may be delisted
Passing...

1 Failed download:
- JFKKU: No data found, symbol may be delisted
Passing...

1 Failed download:
- JFKKW: No data found, symbol may be delisted
Passing...

1 Failed download:
- ACIA: No data found, symbol may be delisted
Passing...

1 Failed download:
- ACAM: No data found, symbol may be delisted
Passing...

1 Failed download:
- ACAMU: No data found, symbol may be delisted
Passing...

1 Failed download:
- ACAMW: No data found, symbol may be delisted
Passing...

1 Failed download:
- ACCP: No data found for this date range, symbol may be delisted
Passing...

1 Failed download:
- XLRN: No data found, symbol ma

ValueError: ignored

In [43]:
model = Sequential()

model.add(Dense(2048, activation='relu', kernel_initializer='he_normal', input_shape=(xTrain.shape[1],)))
model.add(Dense(1024, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(512, activation='relu', kernel_initializer='he_normal'))

#model.add(Dense(2000, activation='relu', kernel_initializer='he_normal'))
#model.add(Dense(1500, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='adam', loss=tf.keras.losses.binary_crossentropy,metrics=['accuracy'])
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_50 (Dense)            (None, 2048)              22528     
                                                                 
 dense_51 (Dense)            (None, 1024)              2098176   
                                                                 
 dense_52 (Dense)            (None, 512)               524800    
                                                                 
 dense_53 (Dense)            (None, 1)                 513       
                                                                 
Total params: 2,646,017
Trainable params: 2,646,017
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.fit(xTrain,yTrain,epochs=25,batch_size=16,validation_data=(xTest,yTest))

Epoch 1/25
8028/8028 [==============================] - 271s 34ms/step - loss: 0.6875 - accuracy: 0.5438 - val_loss: 0.6845 - val_accuracy: 0.5503
Epoch 2/25
8028/8028 [==============================] - 267s 33ms/step - loss: 0.6848 - accuracy: 0.5448 - val_loss: 0.6833 - val_accuracy: 0.5503
Epoch 3/25
8027/8028 [============================>.] - ETA: 0s - loss: 0.6842 - accuracy: 0.5458

KeyboardInterrupt: ignored

In [ ]:
clf = RandomForestClassifier(n_estimators=100)
clf.fit(xTrain,yTrain)
print(f"Random Forest :  {clf.score(xTest,yTest)}")
logisticRegr = LogisticRegression()
logisticRegr.fit(xTrain, yTrain)
print(f"Logistic Regression: {logisticRegr.score(xTest, yTest)}")
nc = NearestCentroid()
nc.fit(xTrain,yTrain)
print(f"Nearest Centroid: {nc.score(xTest,yTest)}")
gnb = GaussianNB()
gnb.fit(xTrain, yTrain)
print(f"GaussianNB: {gnb.score(xTest,yTest)}")
clf2 = KNeighborsClassifier(n_neighbors=2)
clf2.fit(xTrain, yTrain)
print(f"K-Neighbors: {clf2.score(xTest,yTest)}")
tree = DecisionTreeClassifier()
tree.fit(xTrain, yTrain)
print(f"Decision Tree: {tree.score(xTest,yTest)}")
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
   max_depth=1, random_state=0).fit(xTrain, yTrain)
gb.fit(xTrain,yTrain)
print(f"Gradient Boost: {gb.score(xTest, yTest)}")
svm = SVC()
svm.fit(xTrain,yTrain)
print(f"SVM: {svm.score(xTest,yTest)}")